In [ ]:
!pip install pykalman
!pip install pyreadr

In [ ]:
#
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from pykalman import KalmanFilter

data = pd.read_csv('/content/NG.csv')
data = data.dropna(axis=1, how='all')
data = data.dropna(axis=0, how='all')
# Reverse the DataFrame:
data = data.iloc[::-1]
# Reset the index:
data = data.reset_index(drop=True)
# data = data.drop(data.tail(252*2).index)

In [ ]:
#MWH DATA

dt = 1/252
num_simulations = 10000
transportation_cost = 1
discount_rate = 0.05
length = 252*5
correlation_matrix = data['JKM'].tail(length).corr(data['TTF'].tail(length))



def ornstein_uhlenbeck_process(params_JP, params_DE, S0_JP, S0_DE, dt, num_steps, num_simulations, correlation_matrix):
    theta_JP, mu_JP, sigma_JP = params_JP
    theta_DE, mu_DE, sigma_DE = params_DE

    S_t_JP = np.zeros((num_simulations, num_steps))
    S_t_JP[:, 0] = S0_JP

    S_t_DE = np.zeros((num_simulations, num_steps))
    S_t_DE[:, 0] = S0_DE

    # Generate uncorrelated random variables
    uncorrelated_random_variables = np.random.normal(size=(num_simulations, num_steps, 2))

    # Use Cholesky decomposition to introduce correlation
    cov = np.array([[1, correlation_matrix],
                    [correlation_matrix, 1]])
    L = np.linalg.cholesky(cov)

    correlated_random_variables = np.matmul(uncorrelated_random_variables, L.T)

    for i in range(1, num_steps):
        dW_JP = correlated_random_variables[:, i, 0] * np.sqrt(dt)
        S_t_JP[:, i] = S_t_JP[:, i - 1] + theta_JP * (mu_JP - S_t_JP[:, i - 1]) * dt + sigma_JP * dW_JP
        S_t_JP[:, i] = np.maximum(S_t_JP[:, i], 1)  # Set a minimum value of 1e-6

        dW_DE = correlated_random_variables[:, i, 1] * np.sqrt(dt)
        S_t_DE[:, i] = S_t_DE[:, i - 1] + theta_DE * (mu_DE - S_t_DE[:, i - 1]) * dt + sigma_DE * dW_DE
        S_t_DE[:, i] = np.maximum(S_t_DE[:, i], 1)  # Set a minimum value of 1e-6

    return S_t_JP, S_t_DE

# Function to estimate parameters using Kalman Filter
def estimate_parameters(prices, dt):
    # Remove any invalid or missing data points
    prices = prices.dropna()

    # Define the Kalman Filter
    kf = KalmanFilter(transition_matrices=[1],
                      observation_matrices=[1],
                      initial_state_mean=np.mean(prices),
                      initial_state_covariance=np.std(prices),
                      observation_covariance=1,
                      transition_covariance=0.01)

    # Use the observations to estimate the model parameters
    state_means, _ = kf.filter(prices.values)

    # The 'state_means' variable now contains the estimated price levels.
    # We can use these to estimate the parameters of the Ornstein-Uhlenbeck process.
    theta = 1 - state_means[:-1] / state_means[1:]
    mu = state_means[1:] / (1 - theta)
    sigma = np.sqrt(np.var(state_means[1:] - theta * state_means[:-1]))

    # Return the mean of each parameter
    return np.mean(theta), np.mean(mu), np.mean(sigma)

# Function to calculate the payoff of the destination flexibility option
def calculate_option_payoff(prices_JP, prices_DE, transportation_cost):
    return np.maximum(prices_DE - prices_JP - transportation_cost, 0)


def calculate_option_value(params_JP, params_DE, S0_JP, S0_DE, dt, num_steps, num_simulations, transportation_cost, discount_rate, correlation_matrix):
    # Simulate log prices with correlation
    prices_JP, prices_DE = ornstein_uhlenbeck_process(params_JP, params_DE, S0_JP, S0_DE, dt, num_steps, num_simulations, correlation_matrix)

    option_values = []
    option_values_per_step = np.zeros((num_simulations, num_steps))  # Added this line
    for i in range(num_simulations):
        payoff = calculate_option_payoff(prices_JP[i], prices_DE[i], transportation_cost)
        option_value = np.mean(payoff) * np.exp(-discount_rate)
        option_values.append(option_value)
        option_values_per_step[i] = payoff * np.exp(-discount_rate)  # Added this line

    # Calculate the mean option value over all simulations
    mean_option_value = np.mean(option_values)

    return mean_option_value, prices_JP, prices_DE, option_values_per_step  # Modified this line


jp_prices = data['JKM'].tail(length)
de_prices = data['TTF'].tail(length)


theta_de, mu_de, sigma_de = estimate_parameters(de_prices, dt)
theta_jp, mu_jp, sigma_jp = estimate_parameters(jp_prices, dt)

print("Estimated Parameters for DE:")
print("Mean Reversion Strength (theta):", theta_de)
print("Long-Term Mean (mu):", mu_de)
print("Volatility (sigma):", sigma_de)

print("\nEstimated Parameters for JP:")
print("Mean Reversion Strength (theta):", theta_jp)
print("Long-Term Mean (mu):", mu_jp)
print("Volatility (sigma):", sigma_jp)


option_value, prices_JP, prices_DE, option_values_per_step = calculate_option_value(
    [theta_jp, mu_jp, sigma_jp], [theta_de, mu_de, sigma_de],
    jp_prices.iloc[-1],de_prices.iloc[-1],
    dt, length, num_simulations, transportation_cost, discount_rate, correlation_matrix
)


print("Value of the Destination Flexibility Option:", option_value)
print("Value of the Destination Flexibility Option %:", (option_value/jp_prices.iloc[-1])*100)


correlation_JP = np.corrcoef(prices_JP.flatten(), prices_DE.flatten())[0, 1]
print("Correlation between simulated DE and JP prices:", correlation_JP)

prices_JP_flat = pd.Series(prices_JP.flatten())
prices_DE_flat = pd.Series(prices_DE.flatten())



##############PLOTTING#########

jp_actual_prices =  data['JKM'].tail(length).values
de_actual_prices = data['TTF'].tail(length).values

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Find the length of actual prices
len_actual = len(jp_actual_prices)

# Create a grid for the plots
fig = plt.figure(figsize=(20, 8))
gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1])

ax0 = plt.subplot(gs[0])

# Plot actual prices first
ax0.plot(range(len_actual), jp_actual_prices, color='blue', label='Actual JKM Market', linewidth=2)
ax0.plot(range(len_actual), de_actual_prices, color='red', label='Actual TTF Market', linewidth=2)

# Plot simulated prices starting from the end of the actual prices
for i in range(100):
    ax0.plot(range(len_actual, len_actual + len(prices_JP[i])), prices_JP[i], color='blue', alpha=0.05)
    ax0.plot(range(len_actual, len_actual + len(prices_DE[i])), prices_DE[i], color='red', alpha=0.05)


# Plot the specific simulation with a thicker line
random_index = np.random.randint(0, 100)
ax0.plot(range(len_actual, len_actual + len(prices_JP[random_index])), prices_JP[random_index], color='black', linewidth=1, linestyle='--', label='Random JKM Simulation')
ax0.plot(range(len_actual, len_actual + len(prices_DE[random_index])), prices_DE[random_index], color='grey', linewidth=1, linestyle='--', label='Random TTF Simulation')


ax0.set_title('Actual and Simulated Price Paths for Asian (JKM) (blue) and European (TTF) (red) Markets')
ax0.set_xlabel('Time Steps')
ax0.set_ylabel('Price')
ax0.legend()

# Create the distribution plot on the right
ax1 = plt.subplot(gs[1], sharey=ax0)
ax1.hist(prices_JP_flat, color='blue', orientation='horizontal', alpha=0.5, label='JKM Market', bins=50)
ax1.hist(prices_DE_flat, color='red', orientation='horizontal', alpha=0.5, label='TTF Market', bins=50)
ax1.set_xlabel('Frequency')
ax1.legend()

plt.subplots_adjust(wspace=0)
plt.show()